In [1]:
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
from datetime import datetime
from time import sleep, strftime, localtime  
from ib.ext.Contract import Contract
from ib.ext.Order import Order
from ib.opt import Connection, message
import pandas as pd
import backtrader as bt
import pytz, tzlocal
from bs4 import BeautifulSoup
import re
import numpy as np

In [2]:
handler = open('fin.txt').read()
parser = BeautifulSoup(handler, 'lxml')

In [3]:
parser

<html><body><fundamentaldata data="&lt;?xml" encoding="utf-8" reqid="0," version="1.0"><rearnestcons><company><coname><name type="Long">Cryolife Inc</name></coname><coids><coid type="CoId">US_2569</coid><coid type="RepNo">A038A</coid><coid type="IssueId">1</coid><coid type="IssuerID">5919</coid><coid type="RedIssueID">2568</coid><coid type="IsPrimaryIssue">1</coid></coids><securityinfo><security code="1"><exchange code="NYSE">New York Stock Exchange</exchange><country code="USA" set="MX">United States</country><secids><secid set="LOCAL" type="RIC">CRY</secid><secid set="LOCAL" type="Display RIC">CRY.N</secid><secid set="LOCAL" type="TICKER">CRY</secid><secid set="LOCAL" type="ISIN">US2289031005</secid><secid set="LOCAL" type="SEDOL">2239017</secid><secid set="LOCAL" type="CUSIP">228903100</secid></secids><marketdata><marketdataitem currcode="USD" type="CLPRICE" unit="U" updated="2019-08-28T00:00:00">26.890000</marketdataitem><marketdataitem currcode="USD" type="SHARESOUT" unit="U">3749

In [115]:
resc = pd.DataFrame(index = [0],columns = ['Name', 'Exchange', 'Symbol', 'Sector', 'CLPRICE', 'SHARESOUT',
                              'MARKETCAP', '52WKHIGH', '52WKLOW','NAV']

In [118]:
resc

,Name,Exchange,Symbol,Sector,CLPRICE,SHARESOUT,MARKETCAP,52WKHIGH,52WKLOW,NAV as of Aug31 2019
0,Cryolife Inc,New York Stock Exchange,CRY,Healthcare,26.890000,37495602,1008.2567,36.0500,23.9900,1.4200


In [117]:
resc.iloc[:,0] = parser.find('name').text    
resc.iloc[:,1] = parser.find('exchange').text
resc.iloc[:,2] = parser.findAll('secid')[0].text
resc.iloc[:,3] = parser.findAll('sector')[0].text

resc.iloc[:,4] = parser.findAll('marketdataitem')[0].text
resc.iloc[:,5] = parser.findAll('marketdataitem')[1].text
resc.iloc[:,6] = parser.findAll('marketdataitem')[2].text
resc.iloc[:,7] = parser.findAll('marketdataitem')[3].text
resc.iloc[:,8] = parser.findAll('marketdataitem')[4].text
resc.iloc[:,9] = parser.findAll('fyactual')[0].text



In [13]:
parser.findAll('secid')[0]

<secid set="LOCAL" type="RIC">CRY</secid>

In [15]:
parser.findAll('sector')[0]

<sector code="56" set="RBSS2004">Healthcare</sector>

In [188]:
annual = []
columns = []
quarter = []


In [189]:
for i,item in enumerate(parser.findAll('fyactual')):
    columns.append(item['type'])
    for per in item.findAll('fyperiod'):
        if per['periodtype'] == 'A':
            annual.append(per['fyear'])
        if per['periodtype'] == 'Q':
            quarter.append('{}-{}'.format(per['endcalyear'],per['endmonth']))


In [190]:
index = list(set(annual))
index_q = list(set(quarter))

In [191]:
resc_ann = pd.DataFrame(index = index, columns = columns).sort_index(axis = 0,ascending = False)
resc_q = pd.DataFrame(index = index_q, columns = columns).sort_index(axis = 0,ascending = False)

In [192]:
resc_ann

,NAV as of Aug31 2019,CAPEX as of Aug31 2019,NPROFIT as of Aug31 2019,ROA as of Aug31 2019,EPS as of Aug31 2019,REVENUE as of Aug31 2019,EBIT as of Aug31 2019,BVPS as of Aug31 2019,NPROFITREP as of Aug31 2019,PPROFIT as of Aug31 2019,EPSREP as of Aug31 2019,PPROFITREP as of Aug31 2019,EBITDA as of Aug31 2019,NDEBT as of Aug31 2019,ROE as of Aug31 2019
2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [193]:
resc_q.tail()

,NAV as of Aug31 2019,CAPEX as of Aug31 2019,NPROFIT as of Aug31 2019,ROA as of Aug31 2019,EPS as of Aug31 2019,REVENUE as of Aug31 2019,EBIT as of Aug31 2019,BVPS as of Aug31 2019,NPROFITREP as of Aug31 2019,PPROFIT as of Aug31 2019,EPSREP as of Aug31 2019,PPROFITREP as of Aug31 2019,EBITDA as of Aug31 2019,NDEBT as of Aug31 2019,ROE as of Aug31 2019
2014-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [179]:
#datetime.strptime('2018-09', '%Y-%m').strftime('%Y-%m')

In [181]:
resc_q

TypeError: sort_index() got an unexpected keyword argument 'reverse'